<a href="https://colab.research.google.com/github/Jverduzc/CNN_PBX_Model/blob/master/CNN%2BPBX%2B3D%2BNonShock%2B34x32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras
from keras import initializers
from keras.layers import Input, Dropout, BatchNormalization, Conv3DTranspose, concatenate, Dense, Conv3D, Flatten, MaxPooling3D
from keras.models import Sequential
from sklearn.base import BaseEstimator
import keras.backend as K
import pandas as pd
import numpy as np
tf.keras.utils.set_random_seed(0)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sys
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
sys.path.insert(0, '../src/')
import matplotlib.pyplot as plt
import copy
import math

In [ ]:
# TRAINING DATA

paths = ["PBX_PS14,7_Spherical_Oriented_Bottom.npy", "PBX_PS14,7_Spherical_Oriented_Top.npy",
         "PBX_PS14,7_Spherical_Oriented_Bottom_Mirrored.npy", "PBX_PS14,7_Spherical_Oriented_Top_Mirrored.npy",
         
         "PBX_PS9,5_Spherical_Oriented_Bottom.npy", "PBX_PS9,5_Spherical_Oriented_Top.npy",
         "PBX_PS9,5_Spherical_Oriented_Bottom_Mirrored.npy", "PBX_PS9,5_Spherical_Oriented_Top_Mirrored.npy",

         "PBX_PS9,5_Spherical_Random_Bottom.npy", "PBX_PS9,5_Spherical_Random_Top.npy",
         "PBX_PS9,5_Spherical_Random_Bottom_Mirrored.npy", "PBX_PS9,5_Spherical_Random_Top_Mirrored.npy",

         "PBX_PS9,6_Faceted_Random_Bottom.npy", "PBX_PS9,6_Faceted_Random_Top.npy",
         "PBX_PS9,6_Faceted_Random_Bottom_Mirrored.npy", "PBX_PS9,6_Faceted_Random_Top_Mirrored.npy",

         "PBX_PS14,7_Spherical_Random_Bottom_Sim1.npy", "PBX_PS14,7_Spherical_Random_Top_Sim1.npy",
         "PBX_PS14,7_Spherical_Random_Bottom_Mirrored_Sim1.npy", "PBX_PS14,7_Spherical_Random_Top_Mirrored_Sim1.npy"
]

train_data = []
train_labels = []

for i in paths:
  train_ex = np.load("inputs/Inputs_" + i)
  train_lb = np.load("temps/TemperatureMap_" + i)
  train_data.append(train_ex)
  train_labels.append(train_lb)

train_data = np.array(train_data)
train_labels = np.array(train_labels)

print(train_data.shape)
print(train_labels.shape)

In [ ]:
# VALIDATION DATA

validation_paths = [
    "PBX_PS14,7_Spherical_Random_Bottom_Sim2.npy", "PBX_PS14,7_Spherical_Random_Top_Sim2.npy",
    "PBX_PS14,7_Spherical_Random_Bottom_Mirrored_Sim2.npy", "PBX_PS14,7_Spherical_Random_Top_Mirrored_Sim2.npy"
]

validation_data = []
validation_labels = []

for i in validation_paths:
  validation_ex= np.load("inputs/Inputs_" + i)
  validation_lb = np.load("temps/TemperatureMap_" + i)
  validation_data.append(validation_ex)
  validation_labels.append(validation_lb)

validation_data = np.array(validation_data)
validation_labels = np.array(validation_labels)

print(validation_data.shape)
print(validation_labels.shape)

In [ ]:
def periodic_padding_flexible(tensor, axis, padding=1):

    if isinstance(axis,int):
        axis = (axis,)
    if isinstance(padding,int):
        padding = (padding,)

    ndim = len(tensor.shape)

    for ax,p in zip(axis,padding):
        # create a slice object that selects everything from all axes,
        # except only 0:p for the specified for right, and -p: for left

        ind_right = [slice(-p,None) if i == ax else slice(None) for i in range(ndim)]
        ind_left = [slice(0, p) if i == ax else slice(None) for i in range(ndim)]
        right = tensor[ind_right]
        left = tensor[ind_left]
        middle = tensor
        tensor = tf.concat([right,middle,left], axis=ax)

    return tensor


In [ ]:
def DownConvBlock(inputs, n_filters=32, filter_size = 3, max_pooling=True, special_padding=False):

  padding_size = int((filter_size-1)/2)
  kernel_init =   tf.keras.initializers.GlorotUniform(seed=0)
  bias_init = tf.keras.initializers.Zeros()


  # PERIODIC PADDING

  inputs = periodic_padding_flexible(inputs, axis=1,padding=padding_size)
  if special_padding == False:
    inputs = periodic_padding_flexible(inputs, axis=2,padding=padding_size)
  inputs = periodic_padding_flexible(inputs, axis=3,padding=padding_size)

  conv = Conv3D(n_filters, filter_size, activation='relu', padding='valid', kernel_initializer=kernel_init, bias_initializer=bias_init)(inputs)
  print(conv.shape)
  conv = periodic_padding_flexible(conv, axis=(1,2,3),padding=(padding_size,padding_size,padding_size))
  conv = Conv3D(n_filters, filter_size, activation='relu', padding='valid', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv)
  print(conv.shape)
  conv = BatchNormalization()(conv, training=False)
      
  if max_pooling:
    next_layer = tf.keras.layers.MaxPooling3D(pool_size = (2,2,2))(conv)


    # print(conv.shape)
    # next_layer = tf.keras.layers.MaxPooling3D(pool_size = (2,2,2), strides=(1,1,1))(conv)
    # print(next_layer.shape)
    # for i in range(int(conv.shape[1]/2) - 1):
    #   next_layer = tf.keras.layers.MaxPooling3D(pool_size = (2,2,2), strides=(1,1,1))(next_layer)
    #   print(next_layer.shape)
    # for i in range(int(conv.shape[2]/4)):
    #   next_layer = tf.keras.layers.MaxPooling3D(pool_size = (1,2,2), strides=(1,1,1))(next_layer)
    #   print(next_layer.shape)

  else:
    next_layer = conv
  
  skip_connection = conv   

  print("end_of_block") 
  return next_layer, skip_connection

In [ ]:
def UpConvBlock(prev_layer_input, skip_layer_input, filter_size = 3, n_filters=32):

    padding_size = int((filter_size-1)/2)
    kernel_init = tf.keras.initializers.GlorotUniform(seed=0)
    bias_init = tf.keras.initializers.Zeros()   

    up = Conv3DTranspose(n_filters, (filter_size,filter_size,filter_size),
                         strides=(filter_size-1,filter_size-1,filter_size-1),
                         padding='same', kernel_initializer=kernel_init, bias_initializer=bias_init)(prev_layer_input)

    merge = concatenate([up, skip_layer_input], axis=4)
    merge = periodic_padding_flexible(merge, axis=(1,2,3),padding=(padding_size,padding_size,padding_size))
    conv = Conv3D(n_filters, filter_size, activation='relu', padding='valid', kernel_initializer=kernel_init,  bias_initializer=bias_init)(merge)
    print(conv.shape)
    conv = periodic_padding_flexible(conv, axis=(1,2,3),padding=(padding_size,padding_size,padding_size))
    conv = Conv3D(n_filters, filter_size, activation='relu',padding='valid', kernel_initializer=kernel_init,  bias_initializer=bias_init)(conv)
    print(conv.shape)

    print("end_of_block")
    return conv

In [ ]:

def UNet3DModel(input_size=(16, 34, 32, 3), n_filters=32, filter_size=3, n_classes=1):
  kernel_init =  tf.keras.initializers.GlorotUniform(seed=0)
  bias_init = tf.keras.initializers.Zeros()  
  
  inputs = Input(input_size)
  print("Inputs", inputs.shape)

  cblock0 = DownConvBlock(inputs,     n_filters = n_filters    , filter_size = filter_size, max_pooling=False, special_padding=True)
  print("CB0", cblock0[0].shape)

  cblock1 = DownConvBlock(inputs,     n_filters = n_filters    , filter_size = filter_size, max_pooling=True, special_padding=True)
  print("CB1", cblock1[0].shape)

  cblock2 = DownConvBlock(cblock1[0], n_filters = n_filters*2  , filter_size = filter_size, max_pooling=True, special_padding=False)
  print("CB2", cblock2[0].shape)
    
  cblock3 = DownConvBlock(cblock2[0], n_filters = n_filters*4  , filter_size = filter_size, max_pooling=True, special_padding=False)
  print("CB3", cblock3[0].shape)
  
  cblock4 = DownConvBlock(cblock3[0], n_filters = n_filters*8  , filter_size = filter_size, max_pooling=False, special_padding=False)
  print("CB4", cblock4[0].shape)


  print("------------------")

  ublock7 = UpConvBlock(cblock4[0]   , cblock3[1],  n_filters = n_filters * 4, filter_size = filter_size)
  print("UB7", ublock7.shape)
  
  ublock8 = UpConvBlock(ublock7   , cblock2[1],  n_filters = n_filters * 2, filter_size = filter_size)
  print("UB8", ublock8.shape)
  
  ublock9 = UpConvBlock(ublock8   , cblock1[1],  n_filters = n_filters, filter_size = filter_size)
  print("UB9", ublock9.shape)

  ublock9 = periodic_padding_flexible(ublock9, axis=(1,2,3),padding=(1,1,1))
  
  conv9 = Conv3D(n_filters, 3, activation='relu', padding='valid', kernel_initializer=kernel_init,  bias_initializer=bias_init)(ublock9)
  print("C9", conv9.shape)
  
  conv10 = Conv3D(n_classes, 1, padding='same', kernel_initializer=kernel_init,  bias_initializer=bias_init)(conv9)
  print("C10", conv10.shape)

  model = tf.keras.Model(inputs=inputs, outputs=conv10)  

  return model


In [ ]:

def custom_mse(y_true,y_pred):
    w_hot = 5.0
    w_cold = 1.0
    cutoff = 1.8
    weightmat = tf.cast(tf.where(tf.greater(y_true, cutoff), w_hot, w_cold),float)
    loss = tf.cast(K.square(y_pred - y_true),float)
    loss = loss*weightmat
    loss = K.mean(loss)
    return loss

In [ ]:
model = UNet3DModel(input_size=(16, 34, 32, 3), n_filters=32, filter_size = 3, n_classes=1)
optimizer = tf.optimizers.Adam(learning_rate = 0.0005)
model.compile(loss=custom_mse, optimizer=optimizer, metrics=['mse'])

In [ ]:
#model.summary()

In [ ]:
#early = tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0.001, patience=200, verbose=0, mode="auto", baseline=None, restore_best_weights=True)

early = tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0.0005, patience=200, verbose=0, mode="auto", baseline=None, restore_best_weights=True)


def scheduler(epoch, lr):
  if epoch == 2000:
    return lr /5
  else:
    return lr


scheduler_cb = tf.keras.callbacks.LearningRateScheduler(scheduler)


history = model.fit(train_data, train_labels, epochs=4000, validation_data=(validation_data, validation_labels), callbacks=[early, scheduler_cb])

In [ ]:
print(history.history.keys())
# # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.ylim([0,0.2])
plt.legend(['train', 'valid'], loc='upper right')
plt.show()

In [ ]:
def area_hotspot(datapoint):
  tinc = 5
  tnum = int(4500/tinc)

  tinc = 5
  tnum = int(4500/tinc)

  bins=np.zeros((tnum,2))
  bins[:,0] = [ tinc*(0.5 + x) for x in list(range(tnum))]

  for bin_index, temp in np.ndenumerate(datapoint):
    theta = temp * 1000
    tind=math.floor(theta/tinc)
    bins[:tind, 1] += 4 # ~ Roughly 2*2*1 nm^3 (volume value from Chunyu)
  
  return bins

def plot_prediction(data_to_predict, labels_for_data):

  maxval = max(np.max(data_to_predict), np.max(labels_for_data))
  colorscale = [[0, 'rgba(0,0,0,0)'], [0.5, 'rgba(0,0,0,0)'], [1, 'rgb(255,0,0)']]

  fig = make_subplots(rows=2, cols=3, specs=[[{"type": "scatter3d"}, {"type": "histogram"}, {"type": "histogram"}],[{"type": "scatter3d"}, {"type": "scatter"},{"type": "scatter"}]],
    subplot_titles=['Prediction (Temperature)',  'Difference', 'Temp Distribution',  'Labels (Temperature)', 'Parity', 'Volume Plot'], horizontal_spacing = 0.15, vertical_spacing = 0.15)
  

  # FIRST PLOT --> PREDICTIONS

  X,Y,Z = np.mgrid[0:data_to_predict.shape[2], 0:data_to_predict.shape[1], 0:data_to_predict.shape[0]]
  data_to_predict_xz = np.swapaxes(data_to_predict, 2, 0)

  trace_1 = go.Scatter3d(x = X.flatten(), y = Y.flatten(), z=Z.flatten(), hovertemplate = 'Bin: (%{x},%{y},%{z})<br>Temperature: %{marker.color:.2f}<extra></extra>',
                          mode='markers', marker=dict(colorscale=colorscale, symbol='square', color = data_to_predict_xz.flatten(), line=dict(width=0.5, color='rgba(0,0,0,0.025)'),
                                                      cmin=0, cmax=maxval, colorbar=dict(thickness=20, len=0.3, x = 0.27, y= 0.8)), showlegend=False)
  fig.add_trace(trace_1, row=1, col=1)
  ####

  # SECOND PLOT --> LABELS

  X_t,Y_t,Z_t = np.mgrid[0:labels_for_data.shape[2], 0:labels_for_data.shape[1], 0:labels_for_data.shape[0]]
  labels_for_data_xz = np.swapaxes(labels_for_data, 2, 0)
  trace_2 = go.Scatter3d(x = X_t.flatten(), y = Y_t.flatten(), z=Z_t.flatten(), hovertemplate = 'Bin: (%{x},%{y},%{z})<br>Temperature: %{marker.color:.2f}<extra></extra>',
                          mode='markers', marker=dict(colorscale=colorscale, symbol='square', color = labels_for_data_xz.flatten(), line=dict(width=0.5, color='rgba(0,0,0,0.025)'),
                                                      cmin=0, cmax=maxval, colorbar=dict(thickness=20,  len=0.3, x = 0.27, y= 0.3)),showlegend=False)
  fig.add_trace(trace_2, row=2, col=1)
  
  ####

  # TRIRD PLOT
  diff_xz = data_to_predict_xz - labels_for_data_xz
  flat_diff_xz = diff_xz.flatten()

  trace_3 = go.Histogram(x=flat_diff_xz, showlegend=False)
  trace_line = go.Scatter(x=[0,0], y = [0,700], mode='lines', showlegend=False)

  fig.add_trace(trace_3, row=1, col=2)
  fig.add_trace(trace_line, row=1, col=2)


  # FOURTH PLOT
  trace_4 = go.Scatter(x=labels_for_data_xz.flatten(), y = data_to_predict_xz.flatten(), mode='markers', showlegend=False)
  trace_math = go.Scatter(x=[0,maxval], y=[0, maxval], mode='lines', showlegend=False)
  fig.add_trace(trace_4, row=2, col=2)
  fig.add_trace(trace_math, row=2,col=2)
  fig.update_xaxes(title="Scaled Temperature (K) - Labels", row=2, col=2)
  fig.update_yaxes(title="Scaled Temperature (K) - Predictions", row=2, col=2)

  # FIFTH PLOT
  trace_5 = go.Histogram(x=data_to_predict_xz.flatten(), name='Predictions', marker=dict(color='red'), showlegend=True)
  trace_6 = go.Histogram(x=labels_for_data_xz.flatten(), name='Truth', marker=dict(color='green'), showlegend=True)
  fig.update_xaxes(title="Temperature (K) - Labels", row=1, col=3)
  fig.add_trace(trace_5, row=1, col=3)
  fig.add_trace(trace_6, row=1, col=3) 

  # SIXTH PLOT
  
  bins_labels = area_hotspot(labels_for_data)
  bins_predictions = area_hotspot(data_to_predict)
  trace_7 = go.Scatter(x=bins_predictions[:,1], y=bins_predictions[:,0], mode='lines', marker=dict(color='red'), showlegend=False)
  trace_8 = go.Scatter(x=bins_labels[:,1], y=bins_labels[:,0], mode='lines', marker=dict(color='green'), showlegend=False)


  fig.add_trace(trace_7, row=2, col=3)
  fig.add_trace(trace_8, row=2, col=3) 
  fig.update_xaxes(type="log", row=2, col=3)
  fig.update_xaxes(title="Hotspot Volume (nm^3)", row=2, col=3)
  fig.update_yaxes(title="Temperature (K)", row=2, col=3)  

  fig.update_layout(autosize=False, width=1200, height=800, margin=dict(l=75, r=75, b=75, t=75), legend=dict(x=0.85, y=0.25))      
  fig.show()

In [ ]:
### TRAINING DATA

pred_training_data = model.predict(train_data)
#print(pred_training_data.shape)

labels_training_data = train_labels.copy()
#print(labels_training_data.shape)


In [ ]:

for i in [9]:#pred_training_data.shape[0]):
  training_cube = pred_training_data[i].squeeze()
  training_label = labels_training_data[i].squeeze()
  plot_prediction(training_cube, training_label)
  print(" ")


In [ ]:
### VALIDATION DATA

pred_validation_data = model.predict(validation_data)
#print(pred_validation_data.shape)

labels_validation_data = validation_labels.copy()
#print(labels_validation_data.shape)


In [ ]:
for i in [0]:#pred_validation_data.shape[0]):

  validation_cube = pred_validation_data[i].squeeze()
  validation_label = labels_validation_data[i].squeeze()
  plot_prediction(validation_cube, validation_label)

In [ ]:
# TESTING DATA

test_paths = ['/content/drive/MyDrive/PBX/PBXdatasets_small/testPBX6/btm',
              '/content/drive/MyDrive/PBX/PBXdatasets_small/testPBX6/top']

test_data = []
test_labels = []

for i in test_paths:
  test_ex= data_loader_inputs(i)
  test_data.append(test_ex)

test_data = np.array(test_data)
print(test_data.shape)

In [ ]:
def plot_only_prediction(data_to_predict, title):


  maxval = max([np.max(data_to_predict)])

  fig = make_subplots(rows=1, cols=1, specs=[[{"type": "scatter3d"}]], subplot_titles=[title], horizontal_spacing = 0.15, vertical_spacing = 0.15)
  

  # FIRST PLOT --> PREDICTIONS

  X,Y,Z = np.mgrid[0:data_to_predict.shape[2], 0:data_to_predict.shape[1], 0:data_to_predict.shape[0]]
  data_to_predict_xz = np.swapaxes(data_to_predict, 2, 0)

  trace_1 = go.Scatter3d(x = X.flatten(), y = Y.flatten(), z=Z.flatten(),
                          mode='markers', marker=dict(colorscale='Viridis', symbol='square', color = data_to_predict_xz.flatten(),
                                                      cmin=0, cmax=maxval, colorbar=dict(thickness=20)), showlegend=False)
  fig.add_trace(trace_1, row=1, col=1)
  #### 

  fig.update_layout(autosize=False, width=800, height=600, legend=dict(x=0.85, y=0.25))      
  fig.show()

In [ ]:
pred_testing_data = model.predict(test_data)
#print(pred_testing_data.shape)


for i in [0]:#pred_testing_data.shape[0]):

  testing_cube = pred_testing_data[i].squeeze()
  plot_only_prediction(testing_cube, 'Prediction (Temperature)')

In [ ]:
for i in [0]:#pred_testing_data.shape[0]):

  input1_cube = test_data[i,:,:,:,0].squeeze()
  plot_only_prediction(input1_cube, 'HE Density')

  input2_cube = test_data[i,:,:,:,1].squeeze()
  plot_only_prediction(input2_cube, 'Total Density')

  input3_cube = test_data[i,:,:,:,2].squeeze()
  plot_only_prediction(input3_cube, 'GB Interface')